In [806]:
# !/usr/bin/python
# -*- coding: utf-8 -*-
from math import log
from PIL import Image, ImageDraw
from IPython.display import Image

In [807]:
def is_number(s):
    try:
        complex(s) # for int, long, float and complex
    except ValueError:
        return False

    return True

In [808]:
# create ratings.dat dictionary
rentOriginalData = open("./RentData/Kcity02.csv","r")
rentData = []

for rent in rentOriginalData:
    # 去掉\n後，用,區分
    rentData.append(rent.strip().split(','))
#     tempArray = []
#     for value in rent.strip().split(','):
#         if is_number(value):
#             tempArray.append(float(value))
#         else:
#             tempArray.append(value)
                
            
#     rentData.append(tempArray)
#     print (tempArray)

print (len(rentData))

2200


In [809]:
class decisionnode:
  def __init__(self,col=-1,value=None,results=None,tb=None,fb=None):
    self.col = col
    self.value = value
    self.results = results
    self.tb = tb
    self.fb = fb
    
    
# Divides a set on a specific column. Can handle numeric
# or nominal values
def divideset(rows,column,value):
   # Make a function that tells us if a row is in 
   # the first group (true) or the second group (false)
   split_function = None
   if isinstance(value,int) or isinstance(value,float):
      split_function = lambda row:row[column] >= value
   else:
      split_function = lambda row:row[column] == value
   
   # Divide the rows into two sets and return them
   set1=[row for row in rows if split_function(row)]
   set2=[row for row in rows if not split_function(row)]
   return (set1,set2)

In [810]:
divideset(rentData,0,'Lingya')
# divideset(rentData,1,'land-building-car')

([['Lingya',
   'OfficeCommercialBuilding',
   'F',
   'A',
   'F',
   'F',
   'F',
   'F',
   'T',
   'F',
   'F',
   '2300'],
  ['Lingya',
   'OfficeCommercialBuilding',
   'F',
   'A',
   'F',
   'F',
   'F',
   'F',
   'T',
   'F',
   'F',
   '2300'],
  ['Lingya',
   'OfficeCommercialBuilding',
   'F',
   'A',
   'F',
   'F',
   'F',
   'F',
   'T',
   'F',
   'F',
   '2300'],
  ['Lingya',
   'OfficeCommercialBuilding',
   'F',
   'A',
   'F',
   'F',
   'F',
   'F',
   'T',
   'F',
   'F',
   '2400'],
  ['Lingya',
   'OfficeCommercialBuilding',
   'F',
   'A',
   'F',
   'F',
   'F',
   'F',
   'T',
   'T',
   'F',
   '2400'],
  ['Lingya',
   'OfficeCommercialBuilding',
   'F',
   'A',
   'F',
   'F',
   'F',
   'F',
   'T',
   'F',
   'F',
   '2500'],
  ['Lingya',
   'OfficeCommercialBuilding',
   'F',
   'A',
   'F',
   'F',
   'F',
   'F',
   'T',
   'F',
   'F',
   '2500'],
  ['Lingya',
   'OfficeCommercialBuilding',
   'F',
   'A',
   'F',
   'F',
   'F',
   'F',
   'T',
   '

In [811]:
# Create counts of possible results (the last column of 
# each row is the result)
def uniquecounts(rows):
   results={}
   for row in rows:
      # The result is the last column
      r=row[len(row)-1]
      if r not in results: results[r]=0
      results[r]+=1
   return results

In [812]:
# Probability that a randomly placed item will
# be in the wrong category
def giniimpurity(rows):
  total=len(rows)
  counts=uniquecounts(rows)
  imp=0
  for k1 in counts:
    p1=float(counts[k1])/total
    for k2 in counts:
      if k1==k2: continue
      p2=float(counts[k2])/total
      imp+=p1*p2
  return imp

In [813]:
# Entropy is the sum of p(x)log(p(x)) across all 
# the different possible results
def entropy(rows):
   from math import log
   log2=lambda x:log(x)/log(2)  
   results=uniquecounts(rows)
   # Now calculate the entropy
   ent=0.0
   for r in results.keys():
      p=float(results[r])/len(rows)
      ent=ent-p*log2(p)
   return ent

In [814]:
def variance(rows):
  if len(rows)==0: return 0
  data=[float(row[len(row)-1]) for row in rows]
  mean=sum(data)/len(data)
  variance=sum([(d-mean)**2 for d in data])/len(data)
  return variance

In [815]:
giniimpurity(rentData)

0.9767495867770614

In [816]:
entropy(rentData)

6.211751799004848

In [817]:
uniquecounts(rentData)

{'10000': 101,
 '100000': 9,
 '102000': 1,
 '102272': 1,
 '10500': 4,
 '105000': 1,
 '106000': 2,
 '11000': 40,
 '110000': 6,
 '11400': 1,
 '11500': 5,
 '115000': 2,
 '11800': 1,
 '11852': 1,
 '12000': 93,
 '120000': 4,
 '12500': 10,
 '125000': 1,
 '12600': 1,
 '126000': 1,
 '12650': 1,
 '12800': 1,
 '128000': 1,
 '13000': 60,
 '130000': 3,
 '13009': 1,
 '13162': 1,
 '13200': 1,
 '13300': 1,
 '13350': 1,
 '13500': 17,
 '135000': 2,
 '13800': 3,
 '14000': 37,
 '140000': 1,
 '14100': 1,
 '14108': 1,
 '14115': 1,
 '14465': 1,
 '14500': 13,
 '14594': 1,
 '14698': 1,
 '14700': 1,
 '14941': 1,
 '14950': 1,
 '14992': 1,
 '15000': 123,
 '150000': 3,
 '15090': 1,
 '15136': 1,
 '15400': 1,
 '15500': 8,
 '15555': 1,
 '15556': 1,
 '15604': 2,
 '15700': 1,
 '15800': 1,
 '15831': 1,
 '16000': 58,
 '160000': 1,
 '16140': 1,
 '16500': 12,
 '16600': 1,
 '16666': 1,
 '16667': 1,
 '16800': 2,
 '17000': 56,
 '170000': 1,
 '17160': 1,
 '17500': 7,
 '17600': 2,
 '17655': 1,
 '17700': 1,
 '17777': 1,
 '17975

In [818]:
set1,set2=divideset(rentData,0,'Lingya')

In [819]:
entropy(set1)

5.725128755346744

In [820]:
entropy(set2)

6.19665322182111

In [821]:
def buildtree(rows,scoref=giniimpurity):
  if len(rows)==0: return decisionnode()
  current_score=scoref(rows)

  # Set up some variables to track the best criteria
  best_gain=0.0
  best_criteria=None
  best_sets=None
  
  column_count=len(rows[0])-1
  for col in range(0,column_count):
    # Generate the list of different values in
    # this column
    column_values={}
    for row in rows:
       column_values[row[col]]=1
    # Now try dividing the rows up for each value
    # in this column
    for value in column_values.keys():
      (set1,set2)=divideset(rows,col,value)
      
      # Information gain
      p=float(len(set1))/len(rows)
      gain=current_score-p*scoref(set1)-(1-p)*scoref(set2)
      if gain>best_gain and len(set1)>0 and len(set2)>0:
        best_gain=gain
        best_criteria=(col,value)
        best_sets=(set1,set2)
  # Create the sub branches   
  if best_gain>0:
    trueBranch=buildtree(best_sets[0])
    falseBranch=buildtree(best_sets[1])
    return decisionnode(col=best_criteria[0],value=best_criteria[1],
                        tb=trueBranch,fb=falseBranch)
  else:
    return decisionnode(results=uniquecounts(rows))

In [822]:
dtree = buildtree(rentData)

In [823]:
def printtree(tree,indent=''):
   # Is this a leaf node?
   if tree.results!=None:
      print (str(tree.results))
   else:
      # Print the criteria
      print (str(tree.col)+':'+str(tree.value)+'? ')

      # Print the branches
      print (indent+'T->'),
      printtree(tree.tb,indent+'  ')
      print (indent+'F->'),
      printtree(tree.fb,indent+'  ')


def getwidth(tree):
  if tree.tb==None and tree.fb==None: return 1
  return getwidth(tree.tb)+getwidth(tree.fb)

def getdepth(tree):
  if tree.tb==None and tree.fb==None: return 0
  return max(getdepth(tree.tb),getdepth(tree.fb))+1

In [824]:
printtree(dtree)

3:A? 
T->
0:Daliao? 
  T->
{'8000': 5, '10000': 2}
  F->
0:Nanzi? 
    T->
1:Suite? 
      T->
{'4000': 3}
      F->
{'11000': 1}
    F->
6:F? 
      T->
1:OfficeCommercialBuilding? 
        T->
0:Sinsing? 
          T->
{'6000': 1, '6500': 5, '7000': 1, '8500': 2}
          F->
9:F? 
            T->
0:Lingya? 
              T->
{'2300': 3, '2400': 1, '2500': 6, '6000': 1, '6500': 1, '7000': 3, '7500': 1, '8000': 2, '8500': 1, '9000': 2, '10000': 1}
              F->
{'6000': 1}
            F->
4:F? 
              T->
{'2400': 1}
              F->
{'8500': 1}
        F->
9:T? 
          T->
1:Townhouse? 
            T->
0:Qianzhen? 
              T->
{'7000': 1}
              F->
0:Sinsing? 
                T->
{'8000': 1}
                F->
0:Zuoying? 
                  T->
{'18000': 1}
                  F->
0:Sanmin? 
                    T->
{'20000': 1}
                    F->
{'40000': 1}
            F->
1:ResidentialBuilding? 
              T->
0:Sanmin? 
                T->
{'57

0:Sanmin? 
            T->
9:F? 
              T->
10:F? 
                T->
{'48000': 1, '100000': 1}
                F->
{'48000': 2}
              F->
{'106000': 1}
            F->
8:F? 
              T->
4:T? 
                T->
{'42000': 1}
                F->
{'45000': 1}
              F->
9:F? 
                T->
0:Sinsing? 
                  T->
{'55000': 1}
                  F->
{'58000': 1}
                F->
{'36000': 1}
          F->
0:Lingya? 
            T->
{'20000': 1}
            F->
0:Gushan? 
              T->
{'22000': 1}
              F->
{'30000': 2}
        F->
0:Lingya? 
          T->
2:T? 
            T->
8:T? 
              T->
6:T? 
                T->
10:F? 
                  T->
5:T? 
                    T->
{'20000': 1, '39800': 1, '42000': 2}
                    F->
1:ResidentialBuilding? 
                      T->
{'31500': 1}
                      F->
{'60000': 1}
                  F->
{'52000': 1, '57000': 1, '60000': 1, '85000': 1}
               

                          T->
0:Sanmin? 
                            T->
{'12000': 1, '17000': 1}
                            F->
{'21500': 1}
                          F->
3:C? 
                            T->
0:Daliao? 
                              T->
{'18000': 1}
                              F->
{'23000': 1}
                            F->
0:Sinsing? 
                              T->
{'20000': 1}
                              F->
0:Fengshan? 
                                T->
3:D? 
                                  T->
8:F? 
                                    T->
{'16000': 1}
                                    F->
{'19000': 1}
                                  F->
{'10000': 1}
                                F->
{'19000': 1}
                      F->
3:F? 
                        T->
{'102272': 1}
                        F->
{'20000': 2}
              F->
10:Mechanical? 
                T->
3:G? 
                  T->
0:Zuoying? 
                    T->
{'35000': 2}
        

                                                      T->
3:B? 
                                                        T->
{'8000': 1, '16000': 2, '18000': 1}
                                                        F->
{'15000': 1}
                                                      F->
{'15000': 1}
                                                  F->
{'23000': 1}
                                  F->
0:Gushan? 
                                    T->
5:T? 
                                      T->
3:E? 
                                        T->
{'20000': 1}
                                        F->
4:T? 
                                          T->
3:G? 
                                            T->
{'25000': 1}
                                            F->
{'45000': 1}
                                          F->
{'55000': 1}
                                      F->
3:B? 
                                        T->
{'38000': 1}
                                        F

In [825]:
from PIL import Image,ImageDraw

def drawtree(tree,jpeg='tree.jpg'):
  w=getwidth(tree)*100
  h=getdepth(tree)*100+120

  img=Image.new('RGB',(w,h),(255,255,255))
  draw=ImageDraw.Draw(img)

  drawnode(draw,tree,w/2,20)
  img.save(jpeg,'JPEG')
  
def drawnode(draw,tree,x,y):
  if tree.results==None:
    # Get the width of each branch
    w1=getwidth(tree.fb)*100
    w2=getwidth(tree.tb)*100

    # Determine the total space required by this node
    left=x-(w1+w2)/2
    right=x+(w1+w2)/2

    # Draw the condition string
    draw.text((x-20,y-10),str(tree.col)+':'+str(tree.value),(0,0,0))

    # Draw links to the branches
    draw.line((x,y,left+w1/2,y+100),fill=(255,0,0))
    draw.line((x,y,right-w2/2,y+100),fill=(255,0,0))
    
    # Draw the branch nodes
    drawnode(draw,tree.fb,left+w1/2,y+100)
    drawnode(draw,tree.tb,right-w2/2,y+100)
  else:
    txt=' \n'.join(['%s:%d'%v for v in tree.results.items()])
    draw.text((x-20,y),txt,(0,0,0))

In [826]:
drawtree(dtree,jpeg='treeview.jpg')

OSError: encoder error -2 when writing image file

In [ ]:
from IPython.display import Image
Image(filename='treeview.jpg')